In [1]:
import jsonlines
import pandas as pd
import tokenize_uk
from pandas_profiling import ProfileReport

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB

In [2]:
labels = {
    2: 'Без теми',
    1: 'Коронавірус в Україні',
    3: 'Бойові дії на Донбасі',
    4: 'Справа Шеремета',
    9: 'Справа Гандзюк',
    10: 'MH17',
    11: 'Імпічмент Траму і тиск на Зе'
}

In [3]:
data = []

with jsonlines.open('./annotated_data.json') as reader:
    for obj in reader:
        if obj['annotations']:
            num_label = int(obj['annotations'][0]['label'])
            label = labels[num_label]
            row = {
                'id': obj['id'],
                'text': obj['text'],
                'num_label': num_label,
                'label': label
            }
            data.append(row)

len(data)

1516

In [4]:
df = pd.DataFrame(data)
df

,id,text,num_label,label
0,3047,https://tyzhden.ua/News/235234 . Україна напол...,2,Без теми
1,3048,https://tyzhden.ua/News/235233 . Зеленський за...,2,Без теми
2,3049,https://tyzhden.ua/News/235229 . Регламентний ...,2,Без теми
3,3050,https://tyzhden.ua/News/235230 . Рада прийняла...,2,Без теми
4,3051,https://tyzhden.ua/News/235228 . Звільнений мо...,2,Без теми
...,...,...,...,...
1511,8161,https://tyzhden.ua/News/241649 . У МЗС назвали...,2,Без теми
1512,8162,https://tyzhden.ua/News/241647 . В Україну пов...,2,Без теми
1513,8163,https://tyzhden.ua/News/241645 . Міносвіти про...,1,Коронавірус в Україні
1514,8164,https://tyzhden.ua/News/241646 . Криклій сказа...,1,Коронавірус в Україні


In [5]:
profile = ProfileReport(df)
profile.to_file('profile_report.html')

/opt/anaconda3/lib/python3.7/site-packages/pandas_profiling/base.py:319: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [6]:
X = [row['text'] for row in data]
y = [row['label'] for row in data]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=0, shuffle=True, stratify=y
)

len(X_train), len(X_test), len(y_train), len(y_test)

(1212, 304, 1212, 304)

In [8]:
len(set(y)), len(set(y_train)), len(set(y_test))

(7, 7, 7)

# Baseline

In [9]:
count_vect = CountVectorizer()

train_counts = count_vect.fit_transform(X_train)
print(train_counts.shape)
count_vect.get_feature_names()[5000:5010]

(1212, 24525)


['втікача',
 'втікачем',
 'втілення',
 'втілюють',
 'втім',
 'вуглеводнів',
 'вугілля',
 'вугільної',
 'вузди',
 'вузлів']

In [10]:
clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, y_train, cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

[cross_val] F1: 0.7478898078352482


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [11]:
clf = MultinomialNB().fit(train_counts, y_train)

In [12]:
test_counts = count_vect.transform(X_test)
y_pred = clf.predict(test_counts)

In [13]:
print(classification_report(y_test, y_pred))

                              precision    recall  f1-score   support

                        MH17       1.00      0.75      0.86         4
Імпічмент Траму і тиск на Зе       0.90      1.00      0.95        26
                    Без теми       0.96      0.92      0.94       202
       Бойові дії на Донбасі       0.96      0.98      0.97        50
       Коронавірус в Україні       0.57      0.84      0.68        19
              Справа Гандзюк       1.00      0.50      0.67         2
             Справа Шеремета       0.00      0.00      0.00         1

                    accuracy                           0.92       304
                   macro avg       0.77      0.71      0.72       304
                weighted avg       0.93      0.92      0.92       304



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
